In [ ]:
from astropy.constants import G
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.optimize import minimize

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

from thriftshop.actions import safe_get_actions
from thriftshop.potentials import potentials, galpy_potentials
from thriftshop.config import rsun as ro, vcirc as vo
from thriftshop.galpy_helpers import gala_to_galpy_orbit

In [ ]:
from galpy.actionAngle import estimateDeltaStaeckel

In [ ]:
mw = potentials['fiducial']
bovy_mw = galpy_potentials['fiducial']

In [ ]:
w0s = gd.PhaseSpacePosition(pos=([[-8.122, 0, 0],
                                  [-8.122, 0, 0]]*u.kpc).T,
                            vel=([[15, 245., 20.],
                                  [15, 245., 45.]] * u.km/u.s).T)

orbits = mw.integrate_orbit(
    w0s, dt=0.5*u.Myr, t1=0, t2=6*u.Gyr,
    Integrator=gi.DOPRI853Integrator
)

In [ ]:
from galpy.orbit import Orbit as BovyOrbit

ts = np.linspace(0.,100.,1001)
bovy_o = BovyOrbit([1.,0.1,1.1,0.,0.25])
bovy_o.integrate(ts, bovy_mw)

In [ ]:
n = 1
o = orbits[::10, n]

R = o.cylindrical.rho.to_value(ro)
z = o.cylindrical.z.to_value(ro)
phi = o.cylindrical.phi.to_value(u.rad)

vR = o.cylindrical.v_rho.to_value(vo)
vz = o.cylindrical.v_z.to_value(vo)
vT = (o.cylindrical.rho * o.cylindrical.pm_phi).to_value(vo, u.dimensionless_angles())

In [ ]:
delta = estimateDeltaStaeckel(bovy_mw, R, z)
delta

In [ ]:
from galpy.actionAngle import actionAngleStaeckel
aAS = actionAngleStaeckel(pot=bovy_mw, delta=delta)

In [ ]:
bovy_aaf = aAS.actionsFreqsAngles(R, vR, vT, z, vz, phi)
bovy_aaf = {'actions': bovy_aaf[:3],
            'freqs': bovy_aaf[3:6],
            'angles': bovy_aaf[6:]}

In [ ]:
for i in range(3):
    plt.plot((bovy_aaf['actions'][i] - np.mean(bovy_aaf['actions'][i])) / np.mean(bovy_aaf['actions'][i]))

---

In [ ]:
from thriftshop.config import rsun, vcirc
from thriftshop.galpy_helpers import get_staeckel_actions

In [ ]:
w0s = gd.PhaseSpacePosition(pos=([[-8.122, 0, 0],
                                  [-8.122, 0, 0]]*u.kpc).T,
                            vel=([[25, vcirc.to_value(u.km/u.s), 20.],
                                  [25, vcirc.to_value(u.km/u.s), 40.]]*u.km/u.s).T)

Compare sanders method to Staeckel:

In [ ]:
action_unit = u.kpc * u.km/u.s

In [ ]:
bovy_fiducial_actions = []
for i in range(w0s.shape[0]):
    o = potentials['fiducial'].integrate_orbit(w0s[i], dt=0.5, t1=0, t2=2*u.Gyr)
    bovy_fiducial_actions.append(get_staeckel_actions(o, galpy_potentials['fiducial']))
bovy_fiducial_actions = u.Quantity(bovy_fiducial_actions).to(action_unit)
bovy_fiducial_actions = np.mean(bovy_fiducial_actions, axis=-1)

In [ ]:
fiducial_actions = []
for i in range(w0s.shape[0]):
    aaf = safe_get_actions(potentials['fiducial'], w0s[i])
    fiducial_actions.append(aaf['actions'])
fiducial_actions = u.Quantity(fiducial_actions).to(action_unit)

In [ ]:
(bovy_fiducial_actions - fiducial_actions) / fiducial_actions

---

In [ ]:
def _same_actions_objfunc(p, pos, vy, potential_name, match_actions):
    vx, vz = p
    w0 = gd.PhaseSpacePosition(pos=pos,
                               vel=[vx, vy, vz]*u.km/u.s)
    
    # aaf = safe_get_actions(potential, w0, N_max=8)
    o = potentials[potential_name].integrate_orbit(w0, dt=0.5, t1=0, t2=500*u.Myr)
    actions = get_staeckel_actions(o[::4], galpy_potentials[potential_name]).mean(axis=-1)

    model_actions = actions.to_value(action_unit)
    val = ((model_actions[0] - match_actions[0])**2 + 
           (model_actions[2] - match_actions[2])**2)
    
    return val

In [ ]:
def get_w0s_with_same_actions(fiducial_w0):
    # First, determine actions for the input orbit in the 
    # fiducial potential model. These will be the target action values
    fiducial_actions = []
    for n in range(fiducial_w0.shape[0]):
        o = potentials['fiducial'].integrate_orbit(
            fiducial_w0[n], dt=0.5, t1=0, t2=2*u.Gyr)  # MAGIC NUMBERS
        fiducial_actions.append(
            get_staeckel_actions(o, galpy_potentials['fiducial']))
        
    fiducial_actions = u.Quantity(fiducial_actions).to(u.km/u.s * u.kpc)
    fiducial_actions = np.mean(fiducial_actions, axis=-1)
    
    w0s = {}
    for name in potentials:
        if name == 'fiducial':
            w0s[name] = fiducial_w0
            continue

        w0s[name] = []
        for n in range(fiducial_w0.shape[0]):
            res = minimize(_same_actions_objfunc, 
                           x0=fiducial_w0.v_xyz.value[[0, 2], n],
                           args=(fiducial_w0.pos[n],
                                 vo.to_value(u.km/u.s),
                                 name,
                                 fiducial_actions[n].value),
                           method='powell', 
                           options=dict(maxfev=64))
            
            if res.fun > 1e-3:
                print(f"{name}, {n}: func val = {res.fun} -- Failed to converge")
                    
            w0s[name].append(gd.PhaseSpacePosition(
                pos=fiducial_w0.pos[n],
                vel=[res.x[0], vcirc.value, res.x[1]] * u.km/u.s))
            
        w0s[name] = gd.combine(w0s[name])
    
    return w0s

In [ ]:
fiducial_w0 = gd.PhaseSpacePosition(
    pos=([[-rsun.to_value(u.kpc), 0, 0],
          [-rsun.to_value(u.kpc), 0, 0]]*u.kpc).T,
    vel=([[0, vcirc.to_value(u.km/u.s), 20.],
          [10, vcirc.to_value(u.km/u.s), 45.]]*u.km/u.s).T)

In [ ]:
w0s = get_w0s_with_same_actions(fiducial_w0)

In [ ]:
orbits = {}
for k, w0 in all_w0s.items():
    orbits[k] = potentials[k].integrate_orbit(
        w0, dt=0.5*u.Myr, t1=0, t2=6*u.Gyr
    )

In [ ]:
sorted_keys = sorted(
    orbits.keys(), 
    key=lambda k: potentials[k]['disk'].parameters['m'])

In [ ]:
plot_zlim = 1.75
plot_vzlim = 100

# -----
# vz, z
fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for k, ax in zip(sorted_keys, axes):
    _ = orbits[k].plot(['v_z', 'z'], axes=[ax], 
                       auto_aspect=False, units=[u.km/u.s, u.kpc])
    try:
        ax.set_title(f'${float(k):.1f}' + r' \, {\rm M}_{\rm disk}$')
    except ValueError:
        ax.set_title(f'{k}')

axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[0].set_xlim(-plot_vzlim, plot_vzlim)
axes[0].set_ylim(-plot_zlim, plot_zlim)

fig.tight_layout()

# -----
# R, z

fig, axes = plt.subplots(1, 3, figsize=(15, 5), 
                         sharex=True, sharey=True)

for k, ax in zip(sorted_keys, axes):
    _ = orbits[k].cylindrical.plot(
        ['rho', 'z'], axes=[ax], 
        auto_aspect=False, units=[u.kpc, u.kpc])
    
    try:
        ax.set_title(f'${float(k):.1f}' + r' \, {\rm M}_{\rm disk}$')
    except ValueError:
        ax.set_title(f'{k}')

axes[1].set_ylabel('')
axes[2].set_ylabel('')

axes[0].set_xlim(7., 11.)
axes[0].set_ylim(-plot_zlim, plot_zlim)

fig.tight_layout()

In [ ]:
def objfunc(p, x, vy, potential_name, match_actions):
    vx, vz = p
    w0 = gd.PhaseSpacePosition(pos=[x, 0, 0]*u.kpc,
                               vel=[vx, vy, vz]*u.km/u.s)
    
    # aaf = safe_get_actions(potential, w0, N_max=8)
    o = potentials[potential_name].integrate_orbit(w0, dt=1., t1=0, t2=1*u.Gyr)
    actions = get_staeckel_actions(o, galpy_potentials[potential_name]).mean(axis=-1)

    model_actions = actions.to_value(action_unit)
    val = ((model_actions[0] - match_actions[0])**2 + 
           (model_actions[2] - match_actions[2])**2)
    
    return val

In [ ]:
w0s = gd.PhaseSpacePosition(pos=([[-8.122, 0, 0],
                                  [-8.122, 0, 0]]*u.kpc).T,
                            vel=([[15, vcirc.to_value(u.km/u.s), 20.],
                                  [15, vcirc.to_value(u.km/u.s), 45.]]*u.km/u.s).T)

In [ ]:
bovy_fiducial_actions = []
for i in range(w0s.shape[0]):
    o = potentials['fiducial'].integrate_orbit(w0s[i], dt=0.5, t1=0, t2=2*u.Gyr)
    bovy_fiducial_actions.append(get_staeckel_actions(o, galpy_potentials['fiducial']))
bovy_fiducial_actions = u.Quantity(bovy_fiducial_actions).to(action_unit)
bovy_fiducial_actions = np.mean(bovy_fiducial_actions, axis=-1)

In [ ]:
v0s = {}

v0s['fiducial'] = w0s.v_xyz.T.value

for name in potentials:
    # if name == 'fiducial':
    if name != '1.6':
        continue
        
    v0s[name] = []
    for n in range(w0s.shape[0]):
        res = minimize(objfunc, x0=w0s.v_xyz.value[[0,2], n],
                       args=(-ro.to_value(u.kpc), 
                             vo.to_value(u.km/u.s),
                             name,
                             bovy_fiducial_actions[n].value),
                       method='powell', 
                       options=dict(maxfev=64))
        print(name, n, res.fun)
        v0s[name].append(gd.PhaseSpacePosition(
            pos=[-ro.to_value(u.kpc), 0, 0]*u.kpc,
            vel=[res.x[0], vcirc.value, res.x[1]] * u.km/u.s))

In [ ]:
all_w0s = {}
for k, v0 in v0s.items():
    w0s = gd.PhaseSpacePosition(pos=([[-8.122, 0, 0],
                                      [-8.122, 0, 0]]*u.kpc).T,
                                vel=(v0 * u.km/u.s).T)
    print(k, np.array(v0))
    all_w0s[k] = w0s
    
orbits = {}
for k, w0 in all_w0s.items():
    orbits[k] = potentials[k].integrate_orbit(
        w0, dt=0.5*u.Myr, t1=0, t2=6*u.Gyr
    )